In [1]:
from google.colab import drive
import os
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip /content/drive/MyDrive/Colab/類神經/期末專題/dataset_v2.zip

Archive:  /content/drive/MyDrive/Colab/類神經/期末專題/dataset_v2.zip
   creating: dataset_v2/
   creating: dataset_v2/test/
  inflating: dataset_v2/test/dataset.json  
   creating: dataset_v2/test/img/
  inflating: dataset_v2/test/img/20211206_125104.jpg  
  inflating: dataset_v2/test/img/20211206_125155.jpg  
  inflating: dataset_v2/test/img/20211206_125246.jpg  
  inflating: dataset_v2/test/img/20211206_125336.jpg  
  inflating: dataset_v2/test/img/20211206_125432.jpg  
  inflating: dataset_v2/test/img/20211206_125521.jpg  
  inflating: dataset_v2/test/img/20211206_125629.jpg  
  inflating: dataset_v2/test/img/20211206_130453.jpg  
  inflating: dataset_v2/test/img/20211206_130537.jpg  
  inflating: dataset_v2/test/img/20211206_130623.jpg  
  inflating: dataset_v2/test/img/20211206_130701.jpg  
  inflating: dataset_v2/test/img/20211206_130728.jpg  
  inflating: dataset_v2/test/img/20211206_130909.jpg  
  inflating: dataset_v2/test/img/20211206_131652.jpg  
  inflating: dataset_v2/test/img/2

In [3]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-z_08d2ty
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-z_08d2ty
  Resolved https://github.com/facebookresearch/detectron2.git to commit 06f2892d2c3125deb207fc61f52484a428d67add
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 KB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 KB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 KB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 KB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 KB 16.5 MB/s eta 0:00:

#產生result檔

In [4]:
import pycocotools
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import sys
from io import StringIO
import json
import cv2
import numpy as np
import warnings
warnings.simplefilter("ignore")
#----------------------------------------------#
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
#----------------------------------------------#

# Path to test annotation
path_to_annotation = '/content/dataset_v2/test/dataset.json'
test_img_dir = r'/content/dataset_v2/test/img'

cocoGt = COCO(path_to_annotation)

# Initial your model here
cfg = get_cfg()
cfg.merge_from_file("/content/drive/MyDrive/Colab/類神經/期末專題/output/MaskRCNN_R50_7000/output.yaml")  #output的.yaml檔
cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/Colab/類神經/期末專題/output/MaskRCNN_R50_7000/model_final.pth"  #weight檔

predictor = DefaultPredictor(cfg)

# Predict and save the result to COCO format
summary = []
for _, data in cocoGt.imgs.items():
    test_img_path = os.path.join(test_img_dir, data['file_name'])
    im = cv2.imread(test_img_path)
    
    # Modified by your model prediction method here
    outputs = predictor(im)
    pred = outputs['instances'].to('cpu')

    if len(pred) > 0:
        category = pred.get('pred_classes').numpy()
        pred_masks = pred.get('pred_masks').numpy()
        scores = pred.get('scores').numpy()
        bbox = pred.get('pred_boxes').tensor.numpy()

    for c, m ,s, b in zip(category, pred_masks, scores, bbox):
        m = pycocotools.mask.encode(np.asarray(m, order="F"))
        m['counts'] = m['counts'].decode('utf-8')

        summary.append({
        'image_id': int(data['id']),
        'category_id': int(c) + 1,
        'score': float(s),
        'segmentation': m
        })

with open('/content/drive/MyDrive/Colab/類神經/期末專題/result/result_MRCNN_R50_7000.json', 'w') as json_file:
    json.dump(summary, json_file)

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


In [5]:
import os
file_list = os.listdir('/content/drive/MyDrive/Colab/類神經/期末專題/result')
evaluation_list={}
for name in file_list:
  if 'result' in name:
    data_name = '/content/drive/MyDrive/Colab/類神經/期末專題/result/'+name
    # Load result file
    cocoDt = cocoGt.loadRes(data_name)

    # Evaluate
    cocoEval = COCOeval(cocoGt, cocoDt)
    cocoEval.evaluate()
    cocoEval.accumulate()

    original_stdout = sys.stdout
    string_stdout = StringIO()
    sys.stdout = string_stdout
    cocoEval.summarize()
    sys.stdout = original_stdout

    # Please submit your results
    detail = string_stdout.getvalue()
    evaluation_list[name]=detail

Loading and preparing results...
DONE (t=0.06s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *segm*
DONE (t=0.65s).
Accumulating evaluation results...
DONE (t=0.19s).
Loading and preparing results...
DONE (t=0.52s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *segm*
DONE (t=0.39s).
Accumulating evaluation results...
DONE (t=0.12s).
Loading and preparing results...
DONE (t=0.52s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *segm*
DONE (t=0.50s).
Accumulating evaluation results...
DONE (t=0.14s).
Loading and preparing results...
DONE (t=0.68s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *segm*
DONE (t=0.44s).
Accumulating evaluation results...
DONE (t=0.12s).
Loading and preparing results...
DONE (t=0.48s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *segm*
DONE (t

In [6]:
for key in evaluation_list.keys():
  print('Json檔檔名 : ',key)
  print(evaluation_list[key])
  print("  ")

Json檔檔名 :  result_MRCNN_R50_7000.json
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.659
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.884
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.794
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.118
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.543
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.745
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.657
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.736
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.736
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.237
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.651
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.807

 